# Setting up Y_Bus, bus and line objects for all task

In [4]:
from classes import *
import pandas as pd
import os

pathToFile = os.path.abspath(os.curdir)
df_line_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Line_data")
df_bus_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Bus_data")
line_data = df_line_data.to_dict(orient='records')
bus_data = df_bus_data.to_dict(orient='records')
line_admittences = []
num_buses = len(bus_data)

for line_dict in line_data:
    line = Line(
        bus_p=line_dict['From line'],
        bus_q=line_dict['To line'],
        impedance=complex(line_dict['R[pu]'], line_dict['X[pu]']),
        half_line_charging_admittance=complex(line_dict['Half Line Charging Admittance'])
    )
    line_admittences.append(line.build_Line_Y_bus())

# Initialize YBUS for a N-Bus System
Ybus_init = Y_Bus(line_adm=line_admittences)
YBus = Ybus_init.build_YBUS(num_buses=num_buses)
display('YBus', YBus)

'YBus'

,1,2,3,4,5
1,6.305-18.750j,-5.000000+15.000000j,-1.250000+ 3.750000j,0.000000+ 0.000000j,0.00+ 0.00j
2,-5.000+15.000j,10.918333-32.500000j,-1.666667+ 5.000000j,-1.666667+ 5.000000j,-2.50+ 7.50j
3,-1.250+ 3.750j,-1.666667+ 5.000000j,12.971667-38.750000j,-10.000000+30.000000j,0.00+ 0.00j
4,0.000+ 0.000j,-1.666667+ 5.000000j,-10.000000+30.000000j,12.971667-38.750000j,-1.25+ 3.75j
5,0.000+ 0.000j,-2.500000+ 7.500000j,0.000000+ 0.000000j,-1.250000+ 3.750000j,3.79-11.25j


# Newton Raphson Load Flow Algorithm

# Decoupled Load Flow

# Fast Decoupled Load Flow

# DC Power Flow